
The objective of this assignment is to implement and test a k-Nearest Neighbour (k-NN) classifier for time-series data. One of the big benefits of k-NN is that it can be used with data that is not in the normal feature vector format if a distance measure for that data can be found. Dynamic time warping (DTW) is such a measure for time series (DTW Wikipedia page).
Requirements
The notebook Basic_DTW contains some basic code to help you get started.

# Task 1: 1-NN DTW


# dtw Algorithm

In [8]:
import pandas as pd
import numpy as np

In [9]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1,-1]

## 1. Use the DTW function in the notebook to implement a simple 1-NN classifier for time-series data. 

In [10]:
def dtw_1nn(x_train,y_train,x_test, window):
    # calc all date DTW dist
    dtw_distance = []                          # collect DTW distance
    y_test = []                                # collect test lable
    for j in range(len(x_test)):               # loop in X_test
        for i in range(len(x_train)):          # loop in Y_train
            dtw_distance.append(dtw(x_test[j],x_train[i],window))  # put all DTW distance to dtw_distance array
        dtw_distance_index = np.argsort(dtw_distance)  # sort and collect distance index 
        y_test.append(y_train[dtw_distance_index[0]])  # used index[0] to collect Y_traine in min distance 
        dtw_distance.clear()                   # clear dtw_distance data
          
    return y_test


## 2. Test the performance of this classifier on the dataset provided in the file UMD_TEST.txt. This is a three-class synthetic time-series dataset - see details <here>.

In [11]:
# Data Collect 
data = pd.read_csv("./UMD_TEST.txt", sep=r"\s+", header = None)
data.columns = data.columns.astype(str)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,1.0,0.017644,0.030949,0.050555,0.044484,0.053277,0.041576,0.030947,0.027086,0.013764,...,0.024575,0.033780,0.026589,0.013932,0.024928,0.022589,0.038248,0.049838,0.053419,0.040420
1,1.0,0.041296,0.003551,0.027470,0.013158,0.009571,0.008074,0.043743,0.040592,0.012190,...,0.060539,0.046991,0.023586,0.001562,-0.002196,0.036730,0.039027,0.007754,0.004697,0.031440
2,1.0,-0.000720,0.013283,0.029450,0.045201,0.006317,0.018805,0.028901,0.013832,0.015240,...,0.016442,0.039508,0.015171,0.034708,0.010835,0.002942,0.006924,0.029502,0.040786,0.023144
3,1.0,0.005201,0.013363,0.025733,0.026653,0.038946,0.012494,0.028303,0.032011,0.009467,...,0.006383,0.037448,0.044335,0.011143,-0.003624,0.001467,0.020991,0.027675,0.001621,0.015858
4,1.0,0.022926,0.027036,0.011668,0.019500,0.036049,-0.001297,0.019717,0.039583,0.020628,...,0.026997,0.036653,0.018117,0.018314,0.012536,0.040599,0.016590,0.032730,0.002498,0.011260


In [12]:
from sklearn.model_selection import train_test_split

# Data Split  test_size =0.33. and random_state = 42

train_data,test_data = train_test_split(data,test_size=0.33, random_state=42)
y_train=train_data.pop('0').values
x_train=train_data.values
y_test =test_data.pop('0').values
x_test =test_data.values

In [13]:
from sklearn.metrics import accuracy_score
import time

#Predict DTW

dtw_time_start = time.time()        # collect start time to prepare calc capacity
dtw_predict = dtw_1nn(x_train,y_train,x_test,5)
dtw_time_1nn = time.time()-dtw_time_start      # calc run time for the performance

# calc accuracy score of dtw_1nn

dtw_accuracy_score = accuracy_score(y_test, dtw_predict)
print("when test_size =0.33. and random_state = 42 , the performance is")
print('Run Time: %5.2f Sec Accuracy: %5.2f' % (dtw_time_1nn, dtw_accuracy_score))

when test_size =0.33. and random_state = 42 , the performance is
Run Time: 65.36 Sec Accuracy:  0.98


## 3.Given that the time-series are all the same length (150 ticks) Euclidean distance can also be used as distance metric. Compare the 1-NN DTW performance with 1-NN Euclidean. Use the scikit-learn implementation for the Euclidean model. 

In [14]:
from sklearn.metrics.pairwise import euclidean_distances

def euclidean_1nn(x_train,y_train,x_test):
    # calc all date euclidean dist

    euc_distance = []                          # collect ecu distance
    y_test = []                                # collect test lable
    for j in range(len(x_test)):               # loop in X_test
        for i in range(len(x_train)):          # loop in Y_train
                # euclidean_distances refers to the true distance between two points in an m-dimensional space, or the natural length of a vector.
                # we need extended dimension x_test and x_train, also for the distance need covert to float
            euc_distance.append(float(euclidean_distances(np.expand_dims(x_test[j],0),np.expand_dims(x_train[i],0)))) # put all euc distance to dtw_distance array
        euc_distance_index = np.argsort(euc_distance)  # sort and collect distance index
        y_test.append(y_train[euc_distance_index[0]])  # used index[0] to collect Y_traine in min distance
        euc_distance.clear()                   # clear euc_distance data

    return y_test

euc_start = time.time()        # collect start time to prepare calc capacity
euc_predict = euclidean_1nn(x_train,y_train,x_test)
euc_time = time.time()-euc_start
euc_accuracy_score = accuracy_score(y_test, euc_predict)

#### prepare reuslt
result_data = {'Run Time':[euc_time,dtw_time_1nn],'Accuracy':[euc_accuracy_score,dtw_accuracy_score]}
result = pd.DataFrame(result_data, index = ['Euclidean','DTW'])
print("The Result base on test_size =0.33. and random_state = 42")
result


The Result base on test_size =0.33. and random_state = 42


,Run Time,Accuracy
Euclidean,0.297893,0.854167
DTW,65.364594,0.979167


From the results, it can be concluded that Euclidean outperforms DTW in time for the same test_scale, while DTW outperforms Euclidean in accuracy for the best window. The main reason for this is that when Euclidean distance compares two time series, a one-to-one correspondence is established between each point of the series and the sequence, in turn, based on the point-to-point correspondence. The relationship calculates its Euclidean distance as a measure of the distance between the two time series (similarity.) DTW will automatically warp and distort the time series (i.e., locally scale them on the time axis) so that the shapes of the two series are as consistent as possible and the maximum similarity is obtained. Therefore, DTW takes longer time while obtaining higher accuracy.

## 4. Find the best value for the window parameter for this dataset.

In [15]:
# warping cannot be less than the difference in lengths.hence we need find out min Window
# find min_window in dataset
min_window = []
for j in range(len(x_test)):               # loop in X_test
    for i in range(len(x_train)):
       min_window.append(abs(len(x_train[i])-len(x_test[j])))
min_window = max(min_window)
print("The min window of this dataset:" , min_window )

The min window of this dataset: 0


In [17]:
# find best window for this dataset 
dtw_accuracy = []
dtw_time = []
for i in range(min_window,15):
    start = time.time()
    dtw_predict = dtw_1nn(x_train,y_train,x_test,i)
    dtw_time.append(time.time()-start)
    dtw_accuracy.append(accuracy_score(y_test, dtw_predict))

In [19]:
best_window_data = {'Run Time':dtw_time,'Accuracy':dtw_accuracy}
best_window_data = pd.DataFrame(best_window_data)
best_window_data

,Run Time,Accuracy
0,29.362961,0.854167
1,36.782117,0.854167
2,44.008791,0.875000
3,51.180337,0.916667
4,58.243405,0.958333
5,65.497603,0.979167
6,72.090600,0.979167
7,79.144239,0.979167
8,86.157161,0.979167
9,92.738410,0.979167


After we got  that the minimum Window is equal to 0. I used the traditional method to calculate best window.  From  the minimum Window to maximum Window using a For loop. The results are consistent after Windwo = 5 base on dataset test_size = 0.33. and random_state = 42, . So the results can be obtained as follows.

### The Best Window is 5

# Task 2: k-NN DTW

## 1. The k-NN classifier in scikit-learn has a metric parameter that allows for user-defined distance metrics. Adapt the DTW code provided so that it can be incorporated  in a scikit-learn k-NN  classifier. 

In [20]:
def dtw(s, t, window = 5):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0
    
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1,-1]

In [21]:
from sklearn.neighbors import KNeighborsClassifier
estimator = KNeighborsClassifier(n_neighbors=1, metric=dtw)
estimator.fit(x_train,y_train)

KNeighborsClassifier(metric=<function dtw at 0x7f86b15b7f70>, n_neighbors=1)

## 2. Test the performance of this classifier and compare with the 1-NN results from Task 1. Verify that the 1-NN results are consistent. 

In [22]:
import time

# Knn Time and Accuracy
start_time=time.time()
y_predict = estimator.predict(x_test)
end_time=time.time()
cost_time=end_time-start_time
#1NN time and Accuracy
dtw_time_start = time.time()       
dtw_predict = dtw_1nn(x_train,y_train,x_test,5)
dtw_time_1nn = time.time()-dtw_time_start      


dtw_accuracy_score = accuracy_score(y_test, dtw_predict)
knn_dtw_accuracy = accuracy_score(y_test, y_predict)

In [23]:
#### prepare reuslt
result_data = {'Run Time':[cost_time,dtw_time_1nn],'Accuracy':[knn_dtw_accuracy,dtw_accuracy_score]}
result = pd.DataFrame(result_data, index = ['Knn_DTW','1NN_DTW'])
result

,Run Time,Accuracy
Knn_DTW,65.047923,0.979167
1NN_DTW,65.252141,0.979167


##### The results show that Knn-DTW and 1NN- DTW are consistent in accuracy. Since the computer is running other software at the same time, there is a small inconsistent in Run Time, which is negligible.

## 3. Compare with k-NN Euclidean. 

In [24]:
estimator = KNeighborsClassifier(n_neighbors=1)
estimator.fit(x_train,y_train)

#evaluate
start_time=time.time()
y_predict = estimator.predict(x_test)
end_time=time.time()
knn_time=end_time-start_time
knn_accuracy = accuracy_score(y_test, y_predict)

In [25]:
#### prepare reuslt
result_data = {'Run Time':[cost_time,dtw_time_1nn,knn_time,euc_time],'Accuracy':[knn_dtw_accuracy,dtw_accuracy_score, knn_accuracy,euc_accuracy_score]}
result = pd.DataFrame(result_data, index = ['Knn_DTW','1nn_DTW','Knn_Euclidean','1nn_Euclidean'])
result

,Run Time,Accuracy
Knn_DTW,65.047923,0.979167
1nn_DTW,65.252141,0.979167
Knn_Euclidean,0.017984,0.854167
1nn_Euclidean,0.297893,0.854167


##### The results show that Knn-DTW and 1NN- DTW are consistent in accuracy. Since the computer is running other software at the same time, there is a small inconsistent in Run Time, which is negligible.